<a href="https://colab.research.google.com/github/amit-1199/amit-AI/blob/main/text_preprocessing_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Preprocessing Basics (Beginner Tutorial)

This notebook shows **simple, easy-to-read examples** of common text preprocessing steps in Natural Language Processing (NLP).

We will use a small sample text (from a public-style news sentence) and walk through these steps, always showing **before and after**:

- Basic text cleaning (URLs, emojis, punctuation, line breaks, extra spaces, lowercasing)
- Accent & diacritic normalization (e.g., café → cafe)
- Sentence tokenisation
- Word tokenisation
- Subword tokenisation (using a pretrained tokenizer)
- Stop word removal
- Stemming
- Lemmatization
- POS (Part-of-Speech) tagging
- NER (Named Entity Recognition)
- **Identifying and removing outliers** in text datasets

The code is meant for **beginners**, so we avoid clever tricks and keep everything very clear.


In [1]:
pip install jupyter numpy pandas matplotlib nltk spacy transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00


In [2]:
python -m spacy download en_core_web_sm

SyntaxError: invalid syntax (ipython-input-581980377.py, line 1)

In [ ]:
# Install and import libraries (run this cell once)

# If you run this on your own machine, make sure you have installed the packages with:
#  pip install jupyter numpy pandas matplotlib nltk spacy transformers

import re
import unicodedata

import nltk
import spacy
from transformers import AutoTokenizer

# Download small NLTK resources (only needs to be done once per machine)
# These downloads may take a short time.

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("stopwords")

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Load spaCy small English model for POS and NER.
# If you don't have it yet, run this once in a terminal or notebook cell:
#   python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

# Load a simple pretrained tokenizer for subword tokenization
subword_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

print("Libraries are ready.")

In [ ]:
# Our sample text: 3 short paragraphs with emojis, URL, accents and messy spacing
# below is the corpus
sample_text = """I loved visiting the café in Paris 😊!!  The coffee was amazing,
 but the website https://example.com was a bit slow...

Overall, it was a naïve but fun choice 😅!!!   We took many photos 📸📸
 and shared them on social media.   Some people said it was too expensive.

Later, we flew to São Paulo to visit another cafe 😀.   The streets were busy,
 the music was loud, and the desserts were delicious!!!   """

print("ORIGINAL TEXT (RAW):\n")
print(sample_text)

In [ ]:
# Helper: basic cleaning functions with BEFORE / AFTER prints

def remove_urls(text: str) -> str:
    pattern = r"https?://\S+"
    return re.sub(pattern, "", text)


def normalize_accents_and_emojis(text: str) -> str:
    """Convert accents to plain letters and remove emojis/other symbols.

    Steps:
    1. Use Unicode normalization (NFKD) to split accents from letters.
    2. Encode to ASCII and ignore characters we cannot represent (drops emojis).
    3. Decode back to a normal string.
    """

    # Normalize the text to remove accents and emojis (NFKD stands for "Normalization Form
    # KD", which is a way to normalize text by decomposing characters into their base form and then recomposing them)
    normalized = unicodedata.normalize("NFKD", text)

    # Encode to ASCII and ignore characters we cannot represent (drops emojis)
    ascii_bytes = normalized.encode("ascii", "ignore")
    return ascii_bytes.decode("ascii")


def remove_linebreaks(text: str) -> str:
    return text.replace("\n", " ")


def remove_extra_spaces(text: str) -> str:
    return " ".join(text.split())


def remove_punctuation(text: str) -> str:
    # Keep only letters, numbers, and spaces
    return re.sub(r"[^a-zA-Z0-9 ]+", "", text)


def to_lowercase(text: str) -> str:
    return text.lower()


def show_before_after(step_name: str, before: str, after: str) -> None:
    print(f"===== {step_name} =====")
    print("BEFORE:")
    print(before)
    print("\nAFTER:")
    print(after)
    print("\n")

In [ ]:
# 1. Basic cleaning pipeline with BEFORE / AFTER at each step

print("\n==============================")
print("BASIC TEXT CLEANING PIPELINE")
print("==============================\n")

# Start from the raw text with emojis, URL, accents, line breaks, and extra spaces
step1 = remove_urls(sample_text)
show_before_after("Remove URLs", sample_text, step1)

step2 = normalize_accents_and_emojis(step1)
show_before_after("Normalize accents & remove emojis", step1, step2)

step3 = remove_linebreaks(step2)
show_before_after("Remove line breaks", step2, step3)

step4 = remove_extra_spaces(step3)
show_before_after("Remove extra spaces", step3, step4)

step5 = remove_punctuation(step4)
show_before_after("Remove punctuation", step4, step5)

cleaned_text = to_lowercase(step5)
show_before_after("Lowercase", step5, cleaned_text)

print("FINAL CLEANED TEXT (we will use this for tokenisation):\n")
print(cleaned_text)


In [ ]:
nltk.download('punkt_tab')

# 2. Sentence tokenisation (using the original raw text)

print("===== Sentence tokenisation (NLTK) =====\n")
print("Input text (raw):\n")
print(sample_text)

sentences = sent_tokenize(sample_text)

print("\nSentence tokens:\n")
for i, sent in enumerate(sentences, start=1):
    print(f"Sentence {i}: {sent}")


In [ ]:
# 3. Word tokenisation (using the cleaned text)

print("===== Word tokenisation (NLTK) =====\n")
print("Input text (cleaned):\n")
print(cleaned_text)

word_tokens = word_tokenize(cleaned_text)

print("\nFirst 50 word tokens:\n")
print(word_tokens[:50])
print(f"\nTotal number of word tokens: {len(word_tokens)}")

In [ ]:
# 4. Stop word removal

print("===== Stop word removal =====\n")

stop_words = set(stopwords.words("english"))
# stop_words = ['i', 'am', 'the', 'is', 'was']

tokens_no_stop = [token for token in word_tokens if token not in stop_words]

print("Original tokens (first 30):\n")
print(word_tokens[:30])

print("\nAfter removing stop words (first 30):\n")
print(tokens_no_stop[:30])

print(f"\nTotal tokens before: {len(word_tokens)}")
print(f"Total tokens after removing stop words: {len(tokens_no_stop)}")

In [ ]:
# 5. Stemming (PorterStemmer)

print("===== Stemming (PorterStemmer) =====\n")

stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens_no_stop]

print("Original vs stemmed (first 30 tokens):\n")
for original, stem in zip(tokens_no_stop[:30], stemmed_tokens[:30]):
    print(f"{original:15s} -> {stem}")


In [ ]:
# 6. Lemmatization (WordNetLemmatizer)

print("===== Lemmatization (WordNetLemmatizer) =====\n")

lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens_no_stop]

print("Original vs lemmatized (first 30 tokens):\n")
for original, lemma in zip(tokens_no_stop[:30], lemmatized_tokens[:30]):
    print(f"{original:15s} -> {lemma}")

In [ ]:
# 7. POS tagging (Part-of-Speech) and NER (Named Entity Recognition) with spaCy

print("===== POS tagging (spaCy) =====\n")

# Use the original raw text so spaCy can see punctuation and capitalization
pos_doc = nlp(sample_text)

print("Token -> coarse POS tag (pos_) and fine-grained tag (tag_) (first 40 tokens):\n")
for token in pos_doc[:40]:
    print(f"{token.text:15s} {token.pos_:10s} {token.tag_:10s}")

print("\n===== Named Entity Recognition (spaCy) =====\n")

print("Detected entities (text -> label):\n")
for ent in pos_doc.ents:
    print(f"{ent.text:30s} -> {ent.label_}")

In [ ]:
# 8. Subword tokenisation (BERT-style tokenizer)

print("===== Subword tokenisation (BERT tokenizer) =====\n")

# We use the cleaned text here
subword_tokens = subword_tokenizer.tokenize(cleaned_text)

print("First 50 subword tokens:\n")
print(subword_tokens[:50])

input_ids = subword_tokenizer.encode(cleaned_text)
print("\nFirst 50 token IDs:\n")
print(input_ids[:50])

print(f"\nTotal number of subword tokens: {len(subword_tokens)}")


---

## 9. Identifying and Removing Outliers in Text Data

**What are outliers in NLP?**

Outliers in text data are documents that are very different from most other documents. Common examples:
- **Very short texts**: Single words, empty strings, or just a few characters
- **Very long texts**: Extremely long documents that are much longer than average
- **Unusual content**: Texts with mostly special characters, numbers, or gibberish
- **Duplicate texts**: Exact or near-duplicate documents

**Why remove outliers?**

Outliers can:
- Skew your analysis and model training
- Add noise to your dataset
- Make it harder to find patterns in normal data

**How to identify outliers?**

We use simple statistics:
- **Character count**: How many characters are in each document
- **Word count**: How many words are in each document
- **Average word length**: Average length of words in the document

Then we find documents that are too far from the average (using thresholds or statistical methods).


In [ ]:
# Sample dataset: A collection of text documents (some normal, some outliers)

# Normal texts (typical length)
texts = [
    "The weather today is sunny and warm. I went for a walk in the park.",
    "Machine learning is a fascinating field of study. It helps computers learn from data.",
    "I love reading books in my free time. Fiction and non-fiction both interest me.",
    "Cooking is one of my favorite hobbies. I enjoy trying new recipes on weekends.",
    "The city has many beautiful parks and gardens. People love to visit them.",
    "Technology has changed how we communicate. Social media connects people worldwide.",
    "Music brings joy to many people. Different genres appeal to different tastes.",
    "Exercise is important for maintaining good health. Regular activity keeps you fit.",
    "Traveling allows you to experience different cultures. It broadens your perspective.",
    "Education opens doors to many opportunities. Learning never stops throughout life."
]

# Outliers: very short, very long, or unusual content
outlier_texts = [
    "Hi",  # Too short (outlier)
    "",  # Empty (outlier)
    "A",  # Single character (outlier)
    "This is an extremely long text that goes on and on and on. " * 50,  # Very long (outlier)
    "1234567890 !@#$%^&*() 9876543210",  # Mostly numbers and symbols (outlier)
    "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa",  # Repetitive gibberish (outlier)
]

# Combine all texts
all_texts = texts + outlier_texts

print("===== Sample Dataset =====\n")
print(f"Total number of documents: {len(all_texts)}\n")
print("First 3 normal texts:")
for i, text in enumerate(texts[:3], 1):
    print(f"{i}. {text[:60]}...")
print("\nOutlier texts:")
for i, text in enumerate(outlier_texts, 1):
    preview = text[:60] + "..." if len(text) > 60 else text
    print(f"{i}. {preview}")


In [ ]:
# Calculate statistics for each document

import statistics

def calculate_stats(text: str) -> dict:
    """Calculate basic statistics for a text document."""
    char_count = len(text) # count the characters
    words = text.split()
    word_count = len(words) # Count the words
    avg_word_length = statistics.mean([len(word) for word in words]) if words else 0 # average number of words in the sentence
    return {
        "char_count": char_count,
        "word_count": word_count,
        "avg_word_length": avg_word_length
    }

# Calculate stats for all documents
all_stats = [calculate_stats(text) for text in all_texts]

# Calculate average (mean) statistics
avg_char_count = statistics.mean([s["char_count"] for s in all_stats])
avg_word_count = statistics.mean([s["word_count"] for s in all_stats])
avg_word_len = statistics.mean([s["avg_word_length"] for s in all_stats])

# Calculate standard deviation (how spread out the data is)
std_char_count = statistics.stdev([s["char_count"] for s in all_stats]) if len(all_stats) > 1 else 0
std_word_count = statistics.stdev([s["word_count"] for s in all_stats]) if len(all_stats) > 1 else 0

print("===== Document Statistics =====\n")
print("Statistics for each document:\n")
for i, (text, stats) in enumerate(zip(all_texts, all_stats), 1):
    preview = text[:50] + "..." if len(text) > 50 else text
    print(f"Doc {i:2d}: chars={stats['char_count']:4d}, words={stats['word_count']:3d}, "
          f"avg_word_len={stats['avg_word_length']:.1f} | {preview}")

print("\n" + "="*70)
print(f"AVERAGE: chars={avg_char_count:.1f}, words={avg_word_count:.1f}, "
      f"avg_word_len={avg_word_len:.1f}")
print(f"STANDARD DEVIATION: chars={std_char_count:.1f}, words={std_word_count:.1f}")


In [ ]:
# Identify outliers using simple thresholds

# Method 1: Using fixed thresholds (simple and easy to understand)
MIN_CHARS = 10  # Documents with fewer than 10 characters are outliers
MIN_WORDS = 3   # Documents with fewer than 3 words are outliers
MAX_CHARS = 2000  # Documents with more than 2000 characters are outliers

def is_outlier_fixed_threshold(text: str) -> bool:
    """Check if a text is an outlier using fixed thresholds."""
    stats = calculate_stats(text)
    if stats["char_count"] < MIN_CHARS or stats["char_count"] > MAX_CHARS:
        return True
    if stats["word_count"] < MIN_WORDS:
        return True
    return False

# Method 2: Using statistical thresholds (mean ± 2 standard deviations)
# Documents that are more than 2 standard deviations away from the mean are outliers
def is_outlier_statistical(text: str) -> bool:
    """Check if a text is an outlier using statistical method."""
    stats = calculate_stats(text)
    # Check if character count is too far from average
    if abs(stats["char_count"] - avg_char_count) > 2 * std_char_count:
        return True
    # Check if word count is too far from average
    if abs(stats["word_count"] - avg_word_count) > 2 * std_word_count:
        return True
    return False

# Identify outliers using both methods
outliers_fixed = [i for i, text in enumerate(all_texts) if is_outlier_fixed_threshold(text)]
outliers_statistical = [i for i, text in enumerate(all_texts) if is_outlier_statistical(text)]

print("===== Outlier Detection =====\n")
print("Method 1: Fixed Thresholds")
print(f"  Min chars: {MIN_CHARS}, Min words: {MIN_WORDS}, Max chars: {MAX_CHARS}")
print(f"  Outliers found: {outliers_fixed}")
print("\nMethod 2: Statistical (mean ± 2 standard deviations)")
print(f"  Outliers found: {outliers_statistical}\n")

print("Outlier documents identified:\n")
for idx in outliers_fixed:
    preview = all_texts[idx][:60] + "..." if len(all_texts[idx]) > 60 else all_texts[idx]
    stats = all_stats[idx]
    print(f"  Doc {idx+1}: chars={stats['char_count']}, words={stats['word_count']} | {preview}")


In [ ]:
# Remove outliers and show BEFORE / AFTER

print("===== Removing Outliers =====\n")

print("BEFORE (with outliers):")
print(f"  Total documents: {len(all_texts)}")
print(f"  Outliers: {len(outliers_fixed)} documents")
print(f"  Normal documents: {len(all_texts) - len(outliers_fixed)} documents\n")

# Remove outliers (keep only non-outliers)
cleaned_texts = [text for i, text in enumerate(all_texts) if i not in outliers_fixed]
cleaned_indices = [i for i in range(len(all_texts)) if i not in outliers_fixed]

print("AFTER (outliers removed):")
print(f"  Total documents: {len(cleaned_texts)}")
print(f"  Removed: {len(outliers_fixed)} outlier documents\n")

print("Removed outlier documents:")
for idx in outliers_fixed:
    preview = all_texts[idx][:60] + "..." if len(all_texts[idx]) > 60 else all_texts[idx]
    stats = all_stats[idx]
    print(f"  - Doc {idx+1}: chars={stats['char_count']}, words={stats['word_count']} | {preview}")

print("\nKept normal documents (first 5):")
for i, idx in enumerate(cleaned_indices[:5], 1):
    preview = cleaned_texts[i-1][:60] + "..." if len(cleaned_texts[i-1]) > 60 else cleaned_texts[i-1]
    stats = all_stats[idx]
    print(f"  {i}. Doc {idx+1}: chars={stats['char_count']}, words={stats['word_count']} | {preview}")


In [ ]:
# Summary: Statistics comparison BEFORE and AFTER removing outliers

print("===== Summary: Statistics Comparison =====\n")

# Calculate new statistics after removing outliers
cleaned_stats = [calculate_stats(text) for text in cleaned_texts]
cleaned_char_counts = [s["char_count"] for s in cleaned_stats]
cleaned_word_counts = [s["word_count"] for s in cleaned_stats]

print("BEFORE removing outliers:")
print(f"  Average character count: {avg_char_count:.1f}")
print(f"  Average word count: {avg_word_count:.1f}")
print(f"  Character count range: {min([s['char_count'] for s in all_stats])} - {max([s['char_count'] for s in all_stats])}")
print(f"  Word count range: {min([s['word_count'] for s in all_stats])} - {max([s['word_count'] for s in all_stats])}")

print("\nAFTER removing outliers:")
print(f"  Average character count: {statistics.mean(cleaned_char_counts):.1f}")
print(f"  Average word count: {statistics.mean(cleaned_word_counts):.1f}")
print(f"  Character count range: {min(cleaned_char_counts)} - {max(cleaned_char_counts)}")
print(f"  Word count range: {min(cleaned_word_counts)} - {max(cleaned_word_counts)}")

print("\n✅ Outlier removal complete! The dataset is now cleaner and more consistent.")
